## 贝叶斯分类器
**time: 2019/12/25**  
**author: mjl** 


### 1.算法原理
朴素贝叶斯方法是基于贝叶斯定理的一组有监督学习算法， 即“简单”地假设每对特征之间相互独立。
给定一个类别y和$x_{1}$到$x_{n}$的相关的特征向量，贝叶斯定理阐述了以下关系:  
![bys1.png](./img/bys1.png)  
朴素贝叶斯法对条件概率分布作了条件独立性的假设。由于这是一个较强的假设，朴素贝叶斯法也由此得名。具体的，条件独立性假设是:  
![bys2.png](./img/bys2.png)
对于所有的$x_{i}$都成立，这个关系式可以简化为：  
![bys3.png](./img/bys3.png)
由于在给定的输入中$P(x_{1},...,x_{n})$是一个常量，我们使用下面的分类规则:  
![bys4.png](./img/bys4.png)
![bys5.png](./img/bys5.png)
我们可以使用最大后验概率来估计$P(y)$和$P(x_{i}|y)$，前者是训练集中类别y的相对概率。

## 2.高斯朴素贝叶斯分类器
对于高斯朴素贝叶斯分类器，我们假设数据的特征分布为高斯分布， 即:  
![bys6.png](./img/bys6.png)
对于公式（4）我们使用对数函数处理， 便得到如下函数：
![bys7.png](./img/bys7.png)
通过公式（6）， 带入到上述公式得(下面的公式后面那个0.5是多余， 错误写上的)  
![bys8.png](./img/bys8.png)
![bys9.png](./img/bys9.png)
对于高斯朴素贝叶斯分类器的实现就如同上面的公式推导情况一样。对于给定的训练集，我们计算出训练集中每个特征的mean和var，以及每个类别的出现概率。然后对于给定的测试集，我们对每个数据做公式（8）这样的运算， 然后通过公式（9）计算得到最大值，即为该数据所属的类别。

## 3.导包

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import tensorflow as tf
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'
config = tf.ConfigProto()

config.gpu_options.allow_growth = True

session = tf.Session(config=config)

## 4.代码实现

In [23]:
class SGussianNB(object):
    def __init__(self):
        self.epsilon = 1e-9
        pass

    def fit(self, X, Y):
        ## 样本总数量
        counts = X.shape[0]
        ## 将数据按照类别进行划分， 同一个类别的划分到一起
        separate = [[x for x, y in zip(X, Y) if y == c] for c in np.unique(Y)]
        ## 计算每个类别的概率 即使上面的P(y)
        self.class_prob = [np.log(len(i) / counts) for i in separate]
        ## 计算每个类别中， 各个特征的平均值
        self.mean = [np.mean(i, axis=0) for i in separate]
        ## 计算每个类别中， 各个特征的标准差
        self.var = [np.var(i, axis=0) + self.epsilon for i in separate]
        
        self.mean = np.array(self.mean)
        self.var = np.array(self.var)
        self.class_prob = np.array(self.class_prob)
        return self
    
    ## 计算公式8 的后一部分
    def prob_model(self, x,  mean, var):
        exp = -0.5 * np.sum(np.log(2. * np.pi * var))
        exp -= 0.5 * np.sum((x - mean)**2 / var)
        return exp
    
    ## 为每个测试数据， 对于所有的类别计算公式八
    def predict_prob(self, X):
        pre = []
        for x in X:
            temp = []
            for j in range(len(self.mean)):
                temp.append(self.class_prob[j] + self.prob_model(x, self.mean[j], self.var[j]))
            pre.append(temp)
        return np.array(pre)
    
    ## 
    def predict(self, X):
        return np.argmax(self.predict_prob(X), axis=1)

    def score(self, X, Y):
        return np.sum(self.predict(X).reshape([-1, 1]) == Y) / len(Y)

## 5.加载数据

In [24]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

scaler = StandardScaler()
X1 = scaler.fit_transform(load_breast_cancer()['data'])
Y1 = load_breast_cancer()['target'].reshape([-1, 1])
print(X1.shape)
print(Y1.shape)

(569, 30)
(569, 1)


## 6.测试数据

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.1, random_state=0)

print("自己写的：")
nb = SGussianNB().fit(X_train, y_train)
print(nb.score(X_test, y_test))

print("官方提供：")
nb = GaussianNB().fit(X_train, y_train)
print(nb.score(X_test, y_test))

自己写的：
0.894736842105
官方提供：
0.894736842105


/opt/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
